# pipe

> Common operations around the core datastructures for running a sim

In [ ]:
#| default_exp pipe

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from dataclasses import field
from typing import List, Optional, Dict, Callable

from flax.struct import dataclass as flax_dataclass # Using flax's dataclass for JAX-friendliness
import jax.numpy as jnp

# --- Utility Functions ---

def _pad_and_stack_arrays(arrays: List[jnp.ndarray], pad_value: float = jnp.nan) -> jnp.ndarray:
    """
    Pads a list of JAX arrays to a uniform shape and stacks them.

    This function is critical for handling data where entries can have
    variable lengths, such as genetic maps for different chromosomes. JAX's
    core transformations (like `vmap` for vectorization) require arrays to have
    a uniform shape for efficient batching on accelerators (GPU/TPU). This
    utility converts a Python list of arrays into a single, rectangular
    JAX array that meets this requirement.

    Using `jnp.nan` as the default `pad_value` is a robust choice for
    genetic maps, as it's an unambiguous "missing value" marker.
    """
    if not arrays:
        return jnp.array([])
    max_len = max(arr.shape[0] for arr in arrays)
    padded_arrays = [
        jnp.pad(arr, (0, max_len - arr.shape[0]), 'constant', constant_values=pad_value)
        for arr in arrays
    ]
    return jnp.stack(padded_arrays)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()